In [1]:
import pickle

with open('am_dls_data.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data = pickle.load(f)

Format:

(m,n,s, description, loss, mins1, mins2, mins3, A, B, AB)

* `mins1`: Distance from rows of A
* `mins2`: Distance from rows of A (after normalization)
* `mins3`: Distance from rows of B (after normalization)

In [2]:
print(data[0])
len(data[0])

(50, 25.0, 1, '0.05', 0.0012072313201718968, [5.9589126842140085e-05, 2.7439601556850197e-05, 0.031451628032187824, 1.5024873277988663e-05, 1.5733062123442905e-05, 1.6772176030481629e-05, 1.6843647899907493e-05, 1.5465986945487435e-05, 2.0923098679442184e-05, 1.3426876999899373e-05, 1.5754531869910627e-05, 1.2493318459158983e-05, 2.0351709197425065e-05, 1.4583347878382773e-05, 9.0989717067734081e-06, 1.4909880502273148e-05, 1.6095949924939657e-05, 0.1800473828991857, 1.3461020997663213e-05, 1.5361322699899748e-05, 1.9080533910715122e-05, 1.7115748424476458e-05, 1.1755948279215397e-05, 1.814720322407581e-05, 0.38686441714385156, 0.044713727778157651, 0.047713329009351033, 0.06613992698762973, 0.044537292261451852, 0.044503880266226092, 0.058293961057233536, 0.05342464333571819, 0.049554126287625858, 0.037888961171750175, 0.053096245054439203, 0.045692607627229105, 0.053467478598850794, 0.032336818771582947, 0.04966922356517136, 0.052991848518141926, 0.067504227611341325, 0.0457643821992

11

In [2]:
from dl_convergence import *

In [4]:
import numpy as np
from sklearn import datasets, linear_model

def dl_regress(m,s,A,B,size=1000,th=0.5):
    (hs,xs) = make_data_set_pm1(A,size,m,s, setting="h")
    hhs = np.dot(xs,np.transpose(B))
    hhs =  [[x if x>th or x<-th else 0 for x in li] for li in hhs]

    regr1 = linear_model.LinearRegression()
    regr1.fit(hhs, hs)
    
    C1 = regr1.coef_
    score1 = regr1.score(hhs,hs) # is this r^2?
    mse1 = np.mean((regr1.predict(hhs) - hs) ** 2)
    
    regr2 = linear_model.LinearRegression()
    regr2.fit(xs, hs)
    
    C2 = regr2.coef_
    score2 = regr2.score(xs,hs) # is this r^2?
    mse2 = np.mean((regr2.predict(xs) - hs) ** 2)
    
    return (C1, score1, mse1, C2, score2, mse2)


In [ ]:

datar = []
for (m,n,s,st, loss, mins1, mins2, mins3, A, B, AB) in data:
    #print(np.shape(A))
    #print(np.shape(B))
    ( C1, score1, mse1, C2, score2, mse2 ) = dl_regress(m,s,A,B)
    datar.append((m,n,s, st, C1, score1, mse1, C2, score2, mse2))
with open('am_dls_regress.pickle', 'wb') as f:
    pickle.dump(datar, f, pickle.HIGHEST_PROTOCOL)

In [16]:
datar[20]

(50,
 25.0,
 4,
 'oversamples',
 array([[ 0.04454018, -0.30091334, -0.36473386, ..., -0.03839844,
         -0.07511829, -0.04905728],
        [-0.03583482, -0.00746012, -0.02732987, ..., -0.17192393,
          0.04014938,  0.06877415],
        [-0.07933358, -0.04998856, -0.00731774, ...,  0.36774532,
         -0.08746344, -0.03698829],
        ..., 
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]),
 0.5812083832188597,
 0.033477682234744857,
 array([[-0.66966307,  0.60133171,  0.00955092, ..., -0.80367774,
          0.0076848 ,  0.74051583],
        [ 0.58144403, -0.72567755,  0.46479136, ...,  0.99674749,
         -1.21205866,  0.44298387],
        [-0.54511708,  0.32189921, -0.18044952, ..., -0.36536396,
          0.16436484, -0.2

In [5]:
def dl_regress2(m,s,A,B,size=1000,th=0.5):
    (hs,xs) = make_data_set_pm1(A,size,m,s, setting="h")
    hhs = np.dot(xs,np.transpose(B))
    hhs =  [[x if x>th or x<-th else 0 for x in li] for li in hhs]
    (hst,xst) = make_data_set_pm1(A,size,m,s, setting="h")
    hhst = np.dot(xst,np.transpose(B))
    hhst =  [[x if x>th or x<-th else 0 for x in li] for li in hhst]


    regr1 = linear_model.LinearRegression()
    regr1.fit(hhs, hs)
    
    C1 = regr1.coef_
    score1 = regr1.score(hhst,hst) # is this r^2?
    mse1 = np.mean((regr1.predict(hhst) - hst) ** 2)
    
    regr2 = linear_model.LinearRegression()
    regr2.fit(xs, hs)
    
    C2 = regr2.coef_
    score2 = regr2.score(xst,hst) # is this r^2?
    mse2 = np.mean((regr2.predict(xst) - hst) ** 2)
    
    return (C1, score1, mse1, C2, score2, mse2)


In [ ]:

datar2 = []
for (m,n,s,st, loss, mins1, mins2, mins3, A, B, AB) in data:
    #print(np.shape(A))
    #print(np.shape(B))
    ( C1, score1, mse1, C2, score2, mse2 ) = dl_regress(m,s,A,B)
    datar2.append((m,n,s, st, C1, score1, mse1, C2, score2, mse2))
with open('am_dls_regress_test.pickle', 'wb') as f:
    pickle.dump(datar2, f, pickle.HIGHEST_PROTOCOL)

In [23]:
np.set_printoptions(threshold=np.inf,precision=2)
datar2[20][4]
np.shape(datar2[20][4])

(50, 100)

In [30]:
C2 = datar2[19][7]
#np.shape(C2)
C2

array([[ -6.70e-01,   6.01e-01,   9.55e-03,  -3.82e-02,  -3.44e-02,
          5.29e-02,   7.75e-01,  -1.22e-01,  -7.80e-01,  -7.26e-01,
          3.58e-01,   6.98e-01,   2.39e-01,   4.23e-01,   2.00e-01,
          4.61e-02,  -3.48e-01,  -2.29e-01,  -6.08e-01,  -1.71e-01,
         -6.11e-01,  -5.87e-01,  -8.04e-01,   7.68e-03,   7.41e-01],
       [  5.81e-01,  -7.26e-01,   4.65e-01,  -4.61e-01,  -1.02e+00,
         -1.21e-01,  -1.78e-01,   2.39e-01,   3.81e-01,  -7.21e-02,
         -1.35e+00,   5.47e-01,  -1.06e+00,  -9.43e-01,   6.84e-01,
          1.28e+00,   6.18e-01,   7.59e-01,  -1.46e-01,  -2.20e-01,
          2.01e+00,   6.55e-01,   9.97e-01,  -1.21e+00,   4.43e-01],
       [ -5.45e-01,   3.22e-01,  -1.80e-01,   2.92e-01,  -1.77e-01,
          1.10e-01,  -3.45e-01,  -3.60e-01,  -6.67e-01,  -6.18e-01,
         -1.23e-01,  -1.20e+00,   7.57e-01,   1.12e+00,   7.91e-03,
         -1.02e+00,  -8.82e-02,  -6.84e-01,  -4.19e-01,   1.35e-01,
         -1.51e+00,  -7.80e-01,  -3.65e-01,   

In [44]:
i=179
print(datar2[i][0:4])
print(datar2[i][5:7])
print(datar2[i][8:10])

(800, 400.0, 32, 'random')
(0.85800951265412762, 0.0056742128327784552)
(0.99999999922599547, 3.0930708714058576e-11)


In [6]:
m=100
n=50
s=m
A=make_A(m,n)
B=np.transpose(A)
(C1, score1, mse1, C2, score2, mse2) = dl_regress2(m,s,A,B)

KeyboardInterrupt: 

In [1]:
from utils import *

In [6]:
li = sample_without_replacement(100, 100)
print(li)

None


In [26]:
from random import *
def sample_without_replacement(n, k, method="take"):
    if method=="take":
        li = []
        i = 0
        for i in range(k):
            r = randrange(0,n)
            while r in li:
                r = randrange(0,n)
            li.append(r)
        return li
    else:
        li = []
        chosen = 0
        for i in range(n):
            r = random()
            print(r)
            print((k,chosen,n,i))
            print(float(k-chosen)/(n-i))
            if r < float(k-chosen)/(n-i):
                li.append(i)
                chosen = chosen + 1
            if chosen == k:
                break
        return li

In [9]:
sample_without_replacement(100,100)

[33,
 84,
 23,
 30,
 87,
 9,
 1,
 75,
 5,
 32,
 48,
 99,
 54,
 60,
 66,
 61,
 37,
 20,
 88,
 51,
 78,
 70,
 96,
 67,
 26,
 83,
 29,
 94,
 64,
 36,
 25,
 62,
 11,
 21,
 55,
 73,
 42,
 2,
 68,
 79,
 13,
 52,
 6,
 53,
 41,
 58,
 97,
 44,
 74,
 7,
 18,
 71,
 47,
 40,
 89,
 63,
 57,
 90,
 77,
 72,
 10,
 16,
 56,
 46,
 24,
 35,
 17,
 43,
 22,
 19,
 12,
 80,
 15,
 31,
 28,
 76,
 34,
 14,
 8,
 69,
 86,
 45,
 4,
 38,
 81,
 98,
 91,
 50,
 59,
 95,
 49,
 27,
 3,
 93,
 85,
 82,
 92,
 39,
 0,
 65]

In [27]:
sample_without_replacement(100,95,"all")

0.825477559718
(95, 0, 100, 0)
0.95
0.235235954306
(95, 1, 100, 1)
0.949494949495
0.207756114249
(95, 2, 100, 2)
0.948979591837
0.978188453467
(95, 3, 100, 3)
0.948453608247
0.883490348936
(95, 3, 100, 4)
0.958333333333
0.753255594482
(95, 4, 100, 5)
0.957894736842
0.555312698796
(95, 5, 100, 6)
0.957446808511
0.0736963661157
(95, 6, 100, 7)
0.956989247312
0.831924600971
(95, 7, 100, 8)
0.95652173913
0.54273910562
(95, 8, 100, 9)
0.956043956044
0.887266426609
(95, 9, 100, 10)
0.955555555556
0.636059706269
(95, 10, 100, 11)
0.955056179775
0.97137420267
(95, 11, 100, 12)
0.954545454545
0.411504458027
(95, 11, 100, 13)
0.965517241379
0.908520464279
(95, 12, 100, 14)
0.96511627907
0.420900276544
(95, 13, 100, 15)
0.964705882353
0.200253737892
(95, 14, 100, 16)
0.964285714286
0.713801244095
(95, 15, 100, 17)
0.963855421687
0.171517740732
(95, 16, 100, 18)
0.963414634146
0.831056326437
(95, 17, 100, 19)
0.962962962963
0.381294664032
(95, 18, 100, 20)
0.9625
0.456127611649
(95, 19, 100, 21)
0

[0,
 1,
 2,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

In [19]:
random()

0.34992552069296046

In [28]:
import numpy as np
np.ones(5)

array([ 1.,  1.,  1.,  1.,  1.])

In [35]:
np.dot([[1,2],[3,4]],np.ones(2))

array([ 3.,  7.])

In [36]:
x = np.nan

In [37]:
np.isnan(x)

True